In [1]:
import ccxt
import time

# Replace with your actual API keys for Bybit and Bitget
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to Bitget API
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds

# Adjustable trading fees per exchange
TRADING_FEES = {
    'Bybit': 0.001,   # 0.1%
    'Bitget': 0.0008  # 0.08%
}

exchanges = {'Bybit': bybit, 'Bitget': bitget}

def get_tradable_tokens(exchange, exchange_name):
    """Fetches all tradable tokens on an exchange and filters out low-volume pairs"""
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        for symbol, market in markets.items():
            if exchange_name == "Bitget":
                if not market.get('spot', False):  # Ensure it's a spot market
                    continue
            if '/USDT' in symbol and ':USDT' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception as e:
                    print(f"⚠️ Skipping {symbol} on {exchange_name} due to error: {e}")
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    """Fetch latest price for a given token if the market exists on an exchange"""
    try:
        if symbol in exchange.load_markets():
            ticker = exchange.fetch_ticker(symbol)
            return ticker.get("last", None)
    except Exception as e:
        print(f"❌ Error fetching price for {symbol} on {exchange}: {e}")
    return None

def estimate_liquidation_price(entry_price, leverage, direction='long'):
    """Estimates liquidation price for a given entry and leverage (simplified model)"""
    if direction == 'long':
        return entry_price * (1 - 1 / leverage)
    else:  # short
        return entry_price * (1 + 1 / leverage)

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol):
    """Checks for spot vs futures arbitrage opportunities within an exchange"""
    spot_price = get_price(exchange, symbol)
    futures_symbol = symbol.replace('/USDT', '/USDT:USDT')

    if futures_symbol not in exchange.load_markets():
        return

    futures_price = get_price(exchange, futures_symbol)
    if not spot_price or not futures_price:
        return

    trading_fee = TRADING_FEES.get(exchange_name, 0.001)

    spread = (futures_price - spot_price) / spot_price  # Calculate spread %
    net_profit = spread - (trading_fee * 2)  # Adjust for fees

    if net_profit >= SPREAD_THRESHOLD:
        liquidation_price = estimate_liquidation_price(futures_price, leverage=10, direction='short')
        print(f"🔍 Arbitrage Opportunity on {exchange_name}: {symbol}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print(f"    ⚠️ Estimated Liquidation Price (10x short): {liquidation_price:.5f} USDT")
        print(f"    ⚡ Execute arbitrage on {exchange_name}")
        print("-" * 60)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                for token in tradable_tokens:
                    check_intra_exchange_arbitrage(exchange, exchange_name, token)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()


✅ Found 429 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
⚠️ Skipping ISLAND/USDT on Bitget due to error: bitget {"code":"429","msg":"Too Many Requests","requestTime":1742808409480,"data":null}
⚠️ Skipping USUAL/USDT on Bitget due to error: bitget {"code":"429","msg":"Too Many Requests","requestTime":1742808409640,"data":null}
⚠️ Skipping WEETH/USDT on Bitget due to error: bitget {"code":"429","msg":"Too Many Requests","requestTime":1742808411767,"data":null}
⚠️ Skipping MTOS/USDT on Bitget due to error: bitget {"code":"429","msg":"Too Many Requests","requestTime":1742808412605,"data":null}
⚠️ Skipping AI16Z/USDT on Bitget due to error: bitget {"code":"429","msg":"Too Many Requests","requestTime":1742808412722,"data":null}
⚠️ Skipping GPS/USDT on Bitget due to error: bitget {"code":"429","msg":"Too Many Requests","requestTime":1742808412846,"data":null}
⚠️ Skipping CATGOLD/USDT on Bitget due to error: bitget {"code":"429","msg":"Too Many Requests","request